<a href="https://colab.research.google.com/github/dt-cs/Qwen_SFT/blob/main/Qwen3-8b_Retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers transformers datasets accelerate chromadb
!pip install "flash-attn>=2.5.0" --no-build-isolation

In [ ]:
import json

input = "/content/qa_with_metadata.json"

with open(input, "r", encoding="utf-8") as f:
    dataset = json.load(f)

dataset[0]

In [ ]:
# Use 'positive' field as documents
query = [item['question'] for item in dataset]
documents = [item['answer'] for item in dataset]
metadata = [item['metadata'] for item in dataset]

print(query[0])
print(documents[0])
print(query[0])

In [ ]:
!pip install -U sentence-transformers


In [ ]:
# Requires transformers>=4.51.0
# Requires sentence-transformers>=2.7.0

from sentence_transformers import SentenceTransformer


model = SentenceTransformer("Qwen/Qwen3-Embedding-8B")

query_embeddings = model.encode(query)
document_embeddings = model.encode(documents)

# Compute the (cosine) similarity between the query and document embeddings
similarity = model.similarity(query_embeddings, document_embeddings)
print(similarity)
# tensor([[0.7493, 0.0751],
#         [0.0880, 0.6318]])


In [15]:
import chromadb

# Initialize a persistent client by specifying the path
chroma_client = chromadb.PersistentClient(path="./rhino_vector_store")

# Getting or creating a collection remains the same
collection = chroma_client.get_or_create_collection(name="rhino_embeddings")

# Adding documents also remains the same
collection.add(
    documents=documents,
    embeddings=document_embeddings.tolist(),
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"index": i} for i in range(len(documents))]
)

In [25]:
query_01 = "create a box "
query_emb = model.encode(query_01)

results = collection.query(
    query_embeddings=[query_emb],
    n_results=3
)

print(results["documents"][0])



["The 'AddBox' method adds a box-shaped polysurface to the document. You provide a list of 8 corner points. This is useful for quickly creating box geometry:\n\n```python\ncorners = [pt0, pt1, pt2, pt3, pt4, pt5, pt6, pt7]\nbox_id = rs.AddBox(corners)\nprint('Created box:', box_id)\n```", 'The AddRectangle function creates a rectangular curve in the document. You need to specify the plane in which the rectangle lies, as well as its width and height. This is commonly used for creating base shapes for modeling or for layout purposes.', 'The BoundingBox function returns the axis-aligned bounding box of one or more objects. You provide a list of object GUIDs, a view or plane for alignment, and a boolean indicating whether to use world coordinates. This is useful for spatial analysis, collision detection, or layout tasks.']


In [32]:
query_01 = "create a curved surface "
query_emb = model.encode(query_01)

# Compute cosine similarity
similarity = model.similarity(query_emb, document_embeddings)
#print(similarity)
top_idx = torch.argmax(similarity).item()

# Retrieve the best match
print("Best match:", documents[top_idx])

Best match: The AddSpiral function adds a spiral or helical curve to the document. You provide the start and end points, pitch, number of turns, and starting and ending radii. This is useful for creating springs, ramps, or decorative elements. Make sure the parameters are set to match the desired spiral geometry.


In [ ]:
!zip -r /content/rhino_vector_store.zip /content/rhino_vector_store

In [29]:
from google.colab import files
files.download('/content/rhino_vector_store.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>